In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-2.4.5-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [9]:
# load the training data
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [10]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [11]:
lr = LinearRegression(featuresCol='features', labelCol='label', 
                      predictionCol='prediction')

In [12]:
# fit the model
lrModel = lr.fit(training)

In [15]:
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Coefficients: [0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]
Intercept: 0.14228558260358093


In [18]:
trainingSummary = lrModel.summary

In [19]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))

numIterations: 1
objectiveHistory: [0.0]


In [20]:
trainingSummary.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows



In [21]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 10.163092
r2: 0.027839


In [22]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [40]:
# split data -> train 70% and test 30% data
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [41]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 345|
|   mean|-0.07753649432775075|
| stddev|  10.096744337526674|
|    min| -28.046018037776633|
|    max|  24.290551295953957|
+-------+--------------------+



In [42]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                156|
|   mean| 0.9964832721972039|
| stddev| 10.787276807238365|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [43]:
correct_model = lr.fit(train_data)

In [44]:
test_results = correct_model.evaluate(test_data)

In [49]:
test_results.rootMeanSquaredError

10.914515313207902

In [50]:
unlabeled_data = test_data.select('features')

In [51]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [52]:
predictions = correct_model.transform(unlabeled_data)

In [56]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -1.561776274621596|
|(10,[0,1,2,3,4,5,...| -0.6840121294867478|
|(10,[0,1,2,3,4,5,...|  -1.723893211672167|
|(10,[0,1,2,3,4,5,...|  2.5881178404446956|
|(10,[0,1,2,3,4,5,...|   3.431229750843114|
|(10,[0,1,2,3,4,5,...|   0.755844644363073|
|(10,[0,1,2,3,4,5,...|  1.7129672078700178|
|(10,[0,1,2,3,4,5,...|  -2.193231089902323|
|(10,[0,1,2,3,4,5,...|-0.43344623724044995|
|(10,[0,1,2,3,4,5,...|   2.172912990371148|
|(10,[0,1,2,3,4,5,...| -1.0583701290359486|
|(10,[0,1,2,3,4,5,...|  1.7945978126956257|
|(10,[0,1,2,3,4,5,...|  -4.100583985937642|
|(10,[0,1,2,3,4,5,...|  1.1623055445785346|
|(10,[0,1,2,3,4,5,...|   0.947330028955482|
|(10,[0,1,2,3,4,5,...|  1.7861743149906737|
|(10,[0,1,2,3,4,5,...|  0.7726298020599293|
|(10,[0,1,2,3,4,5,...| 0.47260628618836376|
|(10,[0,1,2,3,4,5,...|  1.1560061102693102|
|(10,[0,1,2,3,4,5,...|  -2.57265